# Day 14

## Part 1



In [1]:
# Libraries

import numpy as np
import pandas as pd

# Read input file
all_lines = []

# with open('test_input.txt') as file:
with open('input.txt') as file:
    all_lines = [line.strip() for line in file]
    

In [2]:
# Parse lines
traces = []
for line in all_lines:
    trace = []
    corners_str = line.split(' -> ')
    for corner_str in corners_str:
        x, y = corner_str.split(',')
        x = int(x)
        y = int(y)
        trace.append((x,y))
    
    traces.append(trace)


In [3]:
# Find map edges

all_x = []
all_y = []

for trace in traces:
    for node in trace:
        all_x.append(node[0])
        all_y.append(node[1])

x_min = np.min(all_x)
x_max = np.max(all_x)
y_min = np.min(all_y)
y_max = np.max(all_y)

print(f"x edges: {x_min}, {x_max}")
print(f"y edge: {y_min}, {y_max}")


x edges: 483, 534
y edge: 13, 168


In [4]:
# Create map dataframe

x_map_min = x_min - 2
x_map_max = x_max + 2
y_map_min = 0
y_map_max = y_max + 1

x_coordinates = [x for x in range(x_map_min, x_map_max+1)]
y_coordinates = [y for y in range(y_map_min, y_map_max+1)]

map_df = pd.DataFrame(data='.', index=y_coordinates, columns=x_coordinates)


In [5]:
# Plot trace

from typing import List, Tuple

def plot_trace(*, map_df: pd.DataFrame, trace: List[Tuple[int, int]]) -> pd.DataFrame:
    for node1, node2 in zip(trace[:-1], trace[1:]):
        # Horizontal lines
        if len(set([node1[1], node2[1]])) == 1:
            y = node1[1]
            x_nodes = sorted([node1[0], node2[0]])
            for x in range(x_nodes[0], x_nodes[1]+1):
                map_df.loc[y, x] = '#'

        # Vertical lines
        elif len(set([node1[0], node2[0]])) == 1:
            x = node1[0]
            y_nodes = sorted([node1[1], node2[1]])
            for y in range(y_nodes[0], y_nodes[1]+1):
                map_df.loc[y, x] = '#'
    
    return map_df
        

In [6]:
# Place rocks
for trace in traces:
    map_df = plot_trace(map_df=map_df, trace=trace)
    

In [7]:
# Print map
for idx, row in map_df.iterrows():
    print(''.join(row.values))


........................................................
........................................................
........................................................
........................................................
........................................................
........................................................
........................................................
........................................................
........................................................
........................................................
........................................................
........................................................
........................................................
........................#...............................
............#############...............................
........................................................
........................................................
......................######...

In [8]:
# Move sand method

from typing import Optional, Tuple

def move_sand_unit(
    *, map_df: pd.DataFrame, sand_position: Tuple[int, int]) -> Optional[Tuple[int, int]]:
    x = sand_position[0]
    y = sand_position[1]
    
    down_item = map_df.loc[y+1, x]
    down_left_item = map_df.loc[y+1, x-1]
    down_right_item = map_df.loc[y+1, x+1]
    
    if down_item == '.':
        new_x = x
        new_y = y + 1
    
    else:
        if down_left_item == '.':
            new_x = x - 1
            new_y = y + 1
        
        elif down_right_item == '.':
            new_x = x + 1
            new_y = y + 1
        
        else:
            # No movement possible
            return None
    
    return new_x, new_y
        

In [9]:
# Move sand

sand_start_x = 500
sand_start_y = 0

simulation_map_df = map_df.copy()

y_abyss = y_max + 1

# Simulate sands
n_sand_units = 1

simulation_continue = True
sand_position = (sand_start_x, sand_start_y)

while simulation_continue:
    new_position = move_sand_unit(map_df=simulation_map_df, sand_position=sand_position)
    
    if new_position is not None:
        if new_position[1] >= y_abyss:
            simulation_continue = False
            print('End of simulation')
            print(f'Number of sand units: {n_sand_units}')
            break
        
        sand_position = new_position
        
    else:
        # Final position reached - draw on map and send new sand unit
        simulation_map_df.loc[sand_position[1], sand_position[0]] = 'o'
        n_sand_units = n_sand_units + 1
        sand_position = (sand_start_x, sand_start_y)


End of simulation
Number of sand units: 893


In [10]:
# Print result
print(f'Number of sand units before reaching abyss: {n_sand_units - 1}')

Number of sand units before reaching abyss: 892


---

## Part 2


In [11]:
# Create map dataframe

x_map_min = 500 - y_max - 2 - 2
x_map_max = 500 + y_max + 2 + 2
y_map_min = 0
y_map_max = y_max + 3

x_coordinates = [x for x in range(x_map_min, x_map_max+1)]
y_coordinates = [y for y in range(y_map_min, y_map_max+1)]

p2_map_df = pd.DataFrame(data='.', index=y_coordinates, columns=x_coordinates)


In [12]:
# Place rocks
for trace in traces:
    p2_map_df = plot_trace(map_df=p2_map_df, trace=trace)

# Place floor
floor_trace =[(x_map_min, y_max+2), (x_map_max, y_max+2)]
p2_map_df = plot_trace(map_df=p2_map_df, trace=floor_trace)


In [13]:
# # Print map
# for idx, row in p2_map_df.iterrows():
#     print(''.join(row.values))


In [14]:
# Move sand - Part 2

sand_start_x = 500
sand_start_y = 0

simulation_map_df = p2_map_df.copy()

# Simulate sands
n_sand_units = 1

simulation_continue = True
sand_position = (sand_start_x, sand_start_y)

while simulation_continue:
    new_position = move_sand_unit(map_df=simulation_map_df, sand_position=sand_position)
    
    if new_position is not None:
        sand_position = new_position
    else:
        # Final position reached
        if sand_position == (500, 0):
            simulation_continue = False
            print('End of simulation')
            print(f'Number of sand units: {n_sand_units}')
            break
            
        simulation_map_df.loc[sand_position[1], sand_position[0]] = 'o'
        n_sand_units = n_sand_units + 1
        sand_position = (sand_start_x, sand_start_y)


End of simulation
Number of sand units: 27155


In [15]:
# # Print map
# for idx, row in simulation_map_df.iterrows():
#     print(''.join(row.values))
